# Retention Time Preidiction 

This notebook presents a short walkthrough the process of reading a dataset and training a model for retention time prediction. The dataset is an example dataset extracted from a ProteomTools dataset generated in the **Chair of Bioanalytics** at the **Technical University of Munich**.

The framework being used is a custom wrapper on top of Keras/TensorFlow. The working name of the package is for now `dlpro`.

The available modules in the framework are as follows:

In [ ]:
import pandas as pd

import dlpro
from dlpro import constants, data, eval, layers, models, pipelines, reports, utils
print([x for x in dir(dlpro) if not x.startswith("_")])

- `constants`: constants to be used in the framework (e.g. Aminoacid alphabet mapping)
- `data`:  classes for representing dataset, wrappers around TensorFlow Dataset
- `eval`: custom evaluation metrics implemented in Keras/TF to work as `metrics` for model training
- `layers`: custom layer implementation required for the different models
- `models`: different model implementations for Retention Time Prediction
- `pipelines`: complete pipelines to run a task (e.g. Retention Time prediction)
- `utils`: helper modules

**Note**: reports and pipelines are work-in-progress, some funtionalities are not complete.

## 1. Load Data

We can import the dataset class and create an object of type `RetentionTimeDataset`. This object wraps around TensorFlow dataset objects for training+validation or for testing. This can be specified by the arguments `val_ratio` and `test`.

In [ ]:
from dlpro.data.RetentionTimeDataset import RetentionTimeDataset

In [ ]:
TRAIN_DATAPATH = './example_dataset/proteomTools_train_val.csv'

rtdata = RetentionTimeDataset(data_source=TRAIN_DATAPATH,
                              pad_length=30, batch_size=32, val_ratio=0.2, test=False)

Now we have an RT dataset that can be used directly with standard or custom `Keras` models. This wrapper contains the splits we chose when creating it. In our case, they are training and validation splits. To get the TF Dataset, we call the attributes `.train_data` and `.val_data`.

In [ ]:
len(rtdata.train_data)

In [ ]:
len(rtdata.val_data)

## 2. Model

We can now create the model. We will use a simple Prediction with a conv1D encoder. It has the default working arguments, but most of the parameters can be customized.

**Note**: Important is to ensure that the padding length used for the dataset object is equal to the sequence length passed to the model.

In [ ]:
from dlpro.models.base import RetentionTimePredictor

In [ ]:
model = RetentionTimePredictor(seq_length=30)

## 3. Training

We can then train the model like a standard Keras model. The training parameters here are from Prosit, but other optimizer parameters can be used.  

In [ ]:
#imports

import tensorflow as tf
from dlpro.eval.rt_eval import TimeDeltaMetric

In [ ]:
# create the optimizer object
optimizer = tf.keras.optimizers.Adam(lr=0.0001,decay=1e-7)

# compile the model  with the optimizer and the metrics we want to use, we can add our customer delta95 metric

model.compile(optimizer=optimizer, 
              loss='mse',
              metrics=['mean_absolute_error', TimeDeltaMetric(rtdata.data_mean, rtdata.data_std)])

We store the result of training so that we can explore the metrics and the losses later. We specify the number of epochs for training and pass the training and validation data as previously described.

In [ ]:
history = model.fit(rtdata.train_data,
                    validation_data=rtdata.val_data,
                    epochs=10)

## 3. Testing and Reporting

We can create a test dataset to test our model. Additionally, we can use the reporting module to produce plots and evaluate the model.

Note: the reporting module is still in progress and some functionalities might easily break.

In [ ]:
# create the dataset object for test data

TEST_DATAPATH = './example_dataset/proteomTools_test.csv'

test_rtdata = RetentionTimeDataset(data_source=TEST_DATAPATH,
                              pad_length=30, batch_size=32, test=True)

In [ ]:
# use model.predict from keras directly on the testdata

predictions = model.predict(test_rtdata.test_data)
predictions = predictions.ravel()

In [ ]:
# we can denormalize the targets if we used that when creating the dataset object
# Remeber to use the training data statistics (use the rtdata object)

predictions = rtdata.denormalize_targets(predictions)

In [ ]:
# we can get the targets of a specific split to calcualte evaluation metrics against predictions
# the get_split_targets function from the RetentionTime dataset does this

test_targets = test_rtdata.get_split_targets(split="test")

In [ ]:
test_targets, predictions

In [ ]:
from dlpro.reports.RetentionTimeReport import RetentionTimeReport

In [ ]:
# create a report object by passing the history object and plot different metrics
report = RetentionTimeReport(output_path="./output", history=history)

In [ ]:
report.plot_keras_metric("loss")

In [ ]:
report.plot_keras_metric("mean_absolute_error")

In [ ]:
report.plot_keras_metric("timedelta")

In [ ]:
# calculate R2  given the targets and the predictions of the test data
report.calculate_r2(test_targets, predictions)

In [ ]:
report.plot_highlight_data_portion(test_targets, predictions)

## 4. Saving and Loading Models

Models can be saved normally the same Keras models would be saved. It is better to save the weights and the not the model since it makes it easier and more platform-indepdent when loading the model again. The extra step needed is to create a model object and then load the weights.

In [ ]:
# save the model weights

save_path = "./rtmodel"
model.save_weights(save_path)

In [ ]:
# models can be later loaded by creating a model object and then loading the weights

trained_model = RetentionTimePredictor(seq_length=30)
trained_model.load_weights(save_path)

We can compare the predictions to make sure that the model was loaded correctly.

In [ ]:
new_predictions = trained_model.predict(test_rtdata.test_data)

In [ ]:
new_predictions = rtdata.denormalize_targets(new_predictions)

In [ ]:
# confirm all old and new predictions are the same
(predictions == new_predictions).all()

In [ ]:
predictions = predictions.ravel()

In [ ]:
len(test_rtdata.sequences), len(predictions)

In [ ]:
results_df = pd.DataFrame({"sequence": test_rtdata.sequences,
                           "irt": test_rtdata.targets,
                           "predicted_irt": predictions})

results_df.to_csv("./predictions_irt.csv")